In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from pathlib import Path
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor # Para la magia del paralelismo

BASE_URL = "https://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}
MAX_WORKERS = 10  # Número de hilos simultáneos

# --- UTILIDADES --- (Se mantienen igual o simplificadas)
def safe_float(text):
    num = re.sub(r'[^\d.]', '', text)
    return float(num) if num else 0.0

def rating_to_int(rating_class):
    ratings = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    for word in rating_class:
        if word in ratings: return ratings[word]
    return 0

# --- LÓGICA DE SCRAPING ---

def get_book_detail(session, book_url, cat_name):
    """Procesa un solo libro. Esta función correrá en hilos."""
    try:
        res = session.get(book_url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        
        title = soup.find("h1").get_text(strip=True)
        price = safe_float(soup.find("p", class_="price_color").text)
        
        # Stock: Usamos búsqueda más directa
        stock_text = soup.find("p", class_="instock availability").text
        stock = int(re.search(r'\d+', stock_text).group())
        
        rating = rating_to_int(soup.find("p", class_="star-rating")["class"])
        
        return {
            "Nombre": title,
            "Categoría": cat_name,
            "Precio": price,
            "Stock": stock,
            "Estrellas": rating
        }
    except Exception as e:
        print(f"Error en {book_url}: {e}")
        return None

def scrape_category(cat_name, cat_url):
    """Procesa una categoría entera."""
    category_books = []
    current_url = cat_url
    
    # Usamos una sesión por categoría para eficiencia
    with requests.Session() as session:
        session.headers.update(HEADERS)
        
        while current_url:
            res = session.get(current_url)
            soup = BeautifulSoup(res.text, "html.parser")
            
            # Recolectar todas las URLs de libros de la página actual
            book_tags = soup.find_all("article", class_="product_pod")
            book_urls = [urljoin(current_url, b.find("h3").find("a")["href"]) for b in book_tags]
            
            # Lanzar peticiones de detalles en paralelo para esta página
            with ThreadPoolExecutor(max_workers=5) as executor:
                results = list(executor.map(lambda url: get_book_detail(session, url, cat_name), book_urls))
                category_books.extend([r for r in results if r])
            
            # Siguiente página
            next_btn = soup.find("li", class_="next")
            current_url = urljoin(current_url, next_btn.find("a")["href"]) if next_btn else None
            
    return category_books

def main():
    # 1. Obtener todas las categorías primero
    res = requests.get(BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")
    cat_tags = soup.find("ul", class_="nav-list").find("ul").find_all("a")
    
    tasks = [(t.get_text(strip=True), urljoin(BASE_URL, t['href'])) for t in cat_tags]
    
    print(f"Iniciando scraping de {len(tasks)} categorías con {MAX_WORKERS} hilos...")
    
    all_data = []
    # 2. Procesar categorías en paralelo
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_cat = {executor.submit(scrape_category, name, url): name for name, url in tasks}
        
        for future in future_to_cat:
            all_data.extend(future.result())
            print(f"Finalizado: {future_to_cat[future]}")

    df = pd.DataFrame(all_data)
    print(f"\nTotal: {len(df)} libros.")
    return df

if __name__ == "__main__":
    df_final = main()
    display(df_final.head())

Iniciando scraping de 50 categorías con 10 hilos...
Finalizado: Travel
Finalizado: Mystery
Finalizado: Historical Fiction
Finalizado: Sequential Art
Finalizado: Classics
Finalizado: Philosophy
Finalizado: Romance
Finalizado: Womens Fiction
Finalizado: Fiction
Finalizado: Childrens
Finalizado: Religion
Finalizado: Nonfiction
Finalizado: Music
Finalizado: Default
Finalizado: Science Fiction
Finalizado: Sports and Games
Finalizado: Add a comment
Finalizado: Fantasy
Finalizado: New Adult
Finalizado: Young Adult
Finalizado: Science
Finalizado: Poetry
Finalizado: Paranormal
Finalizado: Art
Finalizado: Psychology
Finalizado: Autobiography
Finalizado: Parenting
Finalizado: Adult Fiction
Finalizado: Humor
Finalizado: Horror
Finalizado: History
Finalizado: Food and Drink
Finalizado: Christian Fiction
Finalizado: Business
Finalizado: Biography
Finalizado: Thriller
Finalizado: Contemporary
Finalizado: Spirituality
Finalizado: Academic
Finalizado: Self Help
Finalizado: Historical
Finalizado: Christ

,Nombre,Categoría,Precio,Stock,Estrellas
0,It's Only the Himalayas,Travel,45.17,19,2
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,49.43,15,4
2,See America: A Celebration of Our National Par...,Travel,48.87,14,3
3,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,8,2
4,Under the Tuscan Sun,Travel,37.33,7,3
